In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

import plot_boundary
import QUBO_SVM_functions as qSVM
import kernel_SVM_functions as kSVM
import metric_functions as mf
from kernel_SVM_functions import rbf_kernel
from classifiers import SoftMarginKernelClassifier, QUBOSoftMarginClassifier


In [2]:
filepath = 'synth_data/synth_0.3.csv'

data = np.loadtxt(filepath, delimiter = ',')
X = data[:, :-1]
t = data[:, -1]

X_train, X_test, t_train, t_test = train_test_split(X, t, train_size = 0.1, shuffle = True, stratify = t)
t_train = t_train.reshape(-1, 1)
t_test = t_test.reshape(-1, 1)

In [3]:
df = pd.read_csv('QA_results/testing_samples.csv')
df

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,29,30,31,32,33,34,35,chain_break_fraction,energy,num_occurrences
0,0,0,0,0,1,1,1,1,1,1,...,1,0,0,0,1,1,1,0.000000,-741.819521,95
1,1,0,0,0,1,1,1,1,1,1,...,1,0,0,0,1,1,1,0.027778,-741.819521,1
2,2,0,0,0,1,1,1,1,1,1,...,1,0,0,0,1,1,1,0.027778,-741.819521,1
3,3,0,0,0,1,1,1,1,1,1,...,1,0,0,0,1,1,1,0.027778,-741.819521,1
4,4,0,0,0,1,1,1,1,1,1,...,1,0,0,0,1,1,1,0.027778,-741.819521,1
5,5,0,0,0,1,1,1,1,1,1,...,1,0,0,0,1,1,1,0.027778,-741.819521,1


In [4]:
alphas = [list(row[1: -3]) for index, row in df.iterrows()]
occurences = [row[-1] for index, row in df.iterrows()]

In [5]:
accuracys = []
aurocs = []

B = 2
K = 3

for encoded_alphas in alphas:
    decoded_alphas = qSVM.decode(np.array(encoded_alphas), B, K)
    print(decoded_alphas)

[0. 7. 7. 7. 0. 0. 0. 0. 7. 7. 0. 7.]
[0. 7. 7. 7. 0. 0. 0. 0. 7. 7. 0. 7.]
[0. 7. 7. 7. 0. 0. 0. 0. 7. 7. 0. 7.]
[0. 7. 7. 7. 0. 0. 0. 0. 7. 7. 0. 7.]
[0. 7. 7. 7. 0. 0. 0. 0. 7. 7. 0. 7.]
[0. 7. 7. 7. 0. 0. 0. 0. 7. 7. 0. 7.]


In [6]:
B = 2
K = 3
R = 1
kernel_func = kSVM.rbf_kernel
gamma = 4

clf = QUBOSoftMarginClassifier(B, K, R, kernel_func, gamma)
clf = clf.make_QUBO_problem(X_train, t_train).fit(X_train, t_train)
clf.alphas

array([[6.],
       [7.],
       [3.],
       [7.],
       [0.],
       [0.],
       [0.],
       [0.],
       [3.],
       [7.],
       [0.],
       [6.]])

In [7]:
clf.ObjectiveScore

-574.3348905757393

In [18]:
slmc = SoftMarginKernelClassifier(7, kernel_func, gamma)
slmc = slmc.fit(X_train, t_train)
d_alphas = slmc.alphas

In [25]:
e_alphas = qSVM.encode_binary(np.array(d_alphas, dtype = int), 3)

In [26]:
clf.ObjectiveFunction(e_alphas)

TypeError: 'QObjectiveFunction' object is not callable